In [38]:
from laptoppriceworkflow.data import get_json_data 
from laptoppriceworkflow.functions import clean_col_meta
from laptoppriceworkflow.functions import simplify_gwarancja
from laptoppriceworkflow.functions import simplify_komunikacja

import pandas as pd

import numpy as np

import seaborn as sns 

import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

In [3]:
df = get_json_data()

In [4]:
msk = np.random.rand(len(df)) < 0.75
train = df[msk]
test = df[~msk]

In [5]:
train.head(2)

,buynow_price,rodzaj karty graficznej,komunikacja,rozdzielczość (piksele),liczba rdzeni procesora,wielkość pamięci ram,system operacyjny,typ dysku twardego,sterowanie,multimedia,typ pamięci ram,taktowanie bazowe procesora (ghz),seria procesora,stan,pojemność dysku (gb),gwarancja,wielkość matrycy
70,1599.0,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,8 gb,[windows 10 home],hdd,"[klawiatura, touchpad]","[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr3,1.9,intel core i3,nowy,500.0,sprzedawcy,"13"" - 13.9"""
1489,2999.0,grafika dedykowana,"[nfc (near field communication), gps]",1920 x 1080,2,12 gb,[windows 10 home],None,[touchpad],"[czytnik kart pamięci, kamera, głośniki, mikrofon]",ddr4,2.0,intel core i3,nowy,480.0,producenta,"15"" - 15.9"""


In [6]:
train_num = train[['taktowanie bazowe procesora (ghz)', 'pojemność dysku (gb)', 'wielkość pamięci ram']]

In [7]:
class FeatureEng(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #imm = joblib.load(PATH_TO_IMM)
        #metadata_dense = reduce_mem_usage(pd.read_csv(METADATA_DENSE).fillna(0))
        #X["country"] = X["city"].map(lambda x: x.split(",")[-1].strip())
        X["rodzaj karty graficznej"].fillna("", inplace=True)
        X["komunikacja"].fillna("", inplace=True)
        X["rozdzielczość (piksele)"].fillna("", inplace=True)
        
        X['liczba_rdzeni_proc_trans'] = ['2' if x == '2' else '' for x in X.loc[:,"liczba rdzeni procesora"]]
        
        X.drop('liczba rdzeni procesora', axis = 1)
        
        X["wielkość pamięci ram"].fillna('0', inplace=True)
        X["wielkosc_pamieci_ram_trans"] = [0 if "mb" in x else int(x.strip('gb')) for x in X.loc[:, "wielkość pamięci ram"]]
        X.drop("wielkość pamięci ram", axis = 1)
        
        X["system operacyjny"].fillna('', inplace=True)
        X["typ dysku twardego"].fillna('', inplace=True)
        
        return X

In [8]:
vectorizer = FeatureEng()
vectorizer.fit_transform(train)
#ddd  = FeatureEng.fit_transform(train)

C:\Users\marcin.sadowski\Miniconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\marcin.sadowski\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\marcin.sadowski\Miniconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,buynow_price,rodzaj karty graficznej,komunikacja,rozdzielczość (piksele),liczba rdzeni procesora,wielkość pamięci ram,system operacyjny,typ dysku twardego,sterowanie,multimedia,typ pamięci ram,taktowanie bazowe procesora (ghz),seria procesora,stan,pojemność dysku (gb),gwarancja,wielkość matrycy,liczba_rdzeni_proc_trans,wielkosc_pamieci_ram_trans
70,1599.00,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,8 gb,[windows 10 home],hdd,"[klawiatura, touchpad]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr3,1.9,intel core i3,nowy,500.0,sprzedawcy,"13"" - 13.9""",2,8
1489,2999.00,grafika dedykowana,"[nfc (near field communication), gps]",1920 x 1080,2,12 gb,[windows 10 home],,[touchpad],"[czytnik kart pamięci, kamera, głośniki, mikro...",ddr4,2.0,intel core i3,nowy,480.0,producenta,"15"" - 15.9""",2,12
407,2999.00,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,16 gb,[windows 7 professional 64-bit],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr3l,2.1,intel pentium dual-core,nowy,480.0,producenta,"15"" - 15.9""",2,16
1359,2849.00,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1920 x 1080,2,8 gb,[windows 10 professional],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr4,2.0,intel core i3,nowy,480.0,producenta,"15"" - 15.9""",2,8
7831,1699.00,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100 mbps]",1366 x 768,4,4 gb,[windows 10 home],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr3l,1.6,intel pentium quad-core,nowy,120.0,producenta,"15"" - 15.9""",,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11321,3212.00,grafika dedykowana,"[bluetooth, lan 10/100/1000 mbps]",1920 x 1080,4,16 gb,[windows 10 home],ssd + hdd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr4,2.6,intel core i7,nowy,1128.0,sprzedawcy,"15"" - 15.9""",,16
11695,2362.23,grafika zintegrowana,"[bluetooth, lan 10/100/1000 mbps]",1920 x 1080,2,4 gb,[windows 10 home],ssd,"[klawiatura, touchpad]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr3,2.3,intel core i5,nowy,256.0,sprzedawcy,"14"" - 14.9""",2,4
10541,1329.00,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,4 gb,[windows 10 home],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr3,1.6,intel celeron dual-core,nowy,128.0,sprzedawcy,"15"" - 15.9""",2,4
8727,4079.00,grafika dedykowana,"[bluetooth, lan 10/100/1000 mbps, lan 10/100 m...",1920 x 1080,4,16 gb,[windows 10 home],ssd + hdd,"[klawiatura, touchpad, klawiatura podświetlana]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr4,2.5,intel core i5,nowy,1128.0,producenta,"15"" - 15.9""",,16


In [8]:
class PandasToNpArray(BaseEstimator, TransformerMixin):
    def fit(self, X, *arg):
        return self

    def transform(self, X):
        return X.values.astype(np.float)

In [25]:
class CustomLabelBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self, sparse_output=False):
        self.sparse_output = sparse_output
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        enc = MultiLabelBinarizer(sparse_output=self.sparse_output)
        return enc.fit_transform(X)

In [28]:
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = MultiLabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

In [51]:
cat_attribs = list(["gwarancja", "typ dysku twardego","rodzaj karty graficznej", "rozdzielczość (piksele)", "liczba rdzeni procesora", 'seria procesora'])
num_attribs = list(train_num)
numerical_features = list(["wielkosc_pamieci_ram_trans"])

#dd = list(['pojemność dysku (gb)', 'gwarancja'])

cat_pipeline = make_pipeline(
        FeatureEng(),
            ColumnTransformer(
            [
                (
                    "cat_vars",
                    make_pipeline(OneHotEncoder(sparse=False)),
                    #list(['gwarancja']),
                    ['rodzaj karty graficznej','gwarancja'],
                ),
                (
                    "ram",
                    make_pipeline(PandasToNpArray(), SimpleImputer(strategy="constant", fill_value=-9999)),
                    numerical_features,
                ),
                (
                    "komunikacja_var",
                    make_pipeline(CountVectorizer(analyzer=lambda x: set(x))),
                    ['komunikacja'],
                ),
            ]
        )
)

In [52]:
tt = cat_pipeline.fit_transform(train)

C:\Users\marcin.sadowski\Miniconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\marcin.sadowski\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\marcin.sadowski\Miniconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 5832 and the array at index 2 has size 1

In [11]:
tt[1]

NameError: name 'tt' is not defined

In [47]:
tt[:,3].mean()

9.25203527815468

In [41]:
mlb = CountVectorizer()
expandedLabelData = mlb.fit_transform(train["komunikacja"][0:10])
#labelClasses = mlb.classes_
#labelClasses
expandedLabelData

AttributeError: 'list' object has no attribute 'lower'

In [50]:
X = [
    ['foo', 'bar'],
    ['bar', 'goo'],
    ['goo', 'foo'],
    ['foo', 'bar', 'goo']
]
#from sklearn.feature_extraction.text import CountVectorizer
x = CountVectorizer(analyzer=set).fit_transform(X)
x

<4x3 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>